In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb
from wideboost.wrappers.wxgb import wxgbModel

from matplotlib import pyplot as plt

In [2]:
import pandas as pd

train = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.data'))
test = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.test'))

In [3]:
ntrain = train.shape[0]
full = np.concatenate([train,test],axis=0)

c0 = full[:,0].astype(float).reshape([-1,1])
c1 = np.asarray(pd.get_dummies(full[:,1]))
c2 = full[:,2].astype(float).reshape([-1,1])
c3 = np.asarray(pd.get_dummies(full[:,3]))
c4 = full[:,4].astype(float).reshape([-1,1])
c5 = np.asarray(pd.get_dummies(full[:,5]))
c6 = np.asarray(pd.get_dummies(full[:,6]))
c7 = np.asarray(pd.get_dummies(full[:,7]))
c8 = np.asarray(pd.get_dummies(full[:,8]))
c9 = np.asarray(pd.get_dummies(full[:,9]))
c10 = full[:,10].astype(float).reshape([-1,1])
c11 = full[:,11].astype(float).reshape([-1,1])
c12 = full[:,12].astype(float).reshape([-1,1])
c13 = np.asarray(pd.get_dummies(full[:,13]))

label = np.zeros([full.shape[0],1])
label[full[:,14] == ' >50K',0] = 1
label[full[:,14] == ' >50K.',0] = 1

XFULL = np.concatenate([c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13],axis=1)
YFULL = label

xtrain = XFULL[0:ntrain,:]
ytrain = label[0:ntrain,0:1]

xtest = XFULL[ntrain:,:]
ytest = label[ntrain:,:]

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    if min(ed1_results['test']['error']) < best_val:
        print("NEW BEST VALUE!")
        best_val = min(ed1_results['test']['error'])
    
    return {'loss': min(ed1_results['test']['error']), 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['binary:logistic']),
    'eval_metric':hp.choice('eval_metric',[['error']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)



{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.8737595633131674, 'colsample_bytree': 0.868407054754693, 'eta': 0.23962603231074467, 'eval_metric': ('error',), 'extra_dims': 8, 'gamma': 5.1162970950730665e-06, 'lambda': 0, 'max_depth': 7, 'min_child_weight': 0.4629783355779059, 'objective': 'binary:logistic', 'subsample': 0.6813388798756006}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[22:05:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you f

/Users/michaelhorrell/github/tensorflow/wideboost/wideboost/objectives/binarylogloss.py:20: RuntimeWarning: overflow encountered in exp
  P = 1 / (1 + np.exp(-logits))



[5]	train-error:0.39975	test-error:0.40319             

[6]	train-error:0.19696	test-error:0.20031             

[7]	train-error:0.62224	test-error:0.62598             

[8]	train-error:0.22319	test-error:0.21855             

[9]	train-error:0.28345	test-error:0.28366             

[10]	train-error:0.23845	test-error:0.23329            

[11]	train-error:0.75918	test-error:0.76376            

[12]	train-error:0.24082	test-error:0.23624            

[13]	train-error:0.24082	test-error:0.23624            

[14]	train-error:0.24082	test-error:0.23624            

[15]	train-error:0.24082	test-error:0.23624            

[16]	train-error:0.75918	test-error:0.76376            

[17]	train-error:0.24082	test-error:0.23624            

[18]	train-error:0.24082	test-error:0.23624            

[19]	train-error:0.24082	test-error:0.23624            

[20]	train-error:0.75918	test-error:0.76376            

[21]	train-error:0.24082	test-error:0.23624            

[22]	train-error:0.24082	test-e

[21]	train-error:0.14097	test-error:0.14288                              

[22]	train-error:0.14122	test-error:0.14288                              

[23]	train-error:0.14088	test-error:0.14257                              

[24]	train-error:0.14060	test-error:0.14251                              

[25]	train-error:0.14045	test-error:0.14263                              

[26]	train-error:0.13983	test-error:0.14183                              

[27]	train-error:0.13965	test-error:0.14183                              

[28]	train-error:0.13983	test-error:0.14202                              

[29]	train-error:0.13965	test-error:0.14165                              

[30]	train-error:0.13968	test-error:0.14140                              

[31]	train-error:0.13959	test-error:0.14091                              

[32]	train-error:0.13953	test-error:0.14072                              

[33]	train-error:0.13944	test-error:0.14054                              

[34]	train-error:0.13928	

[14]	train-error:0.20455	test-error:0.20190                              

[15]	train-error:0.20452	test-error:0.20190                              

[16]	train-error:0.20455	test-error:0.20190                              

[17]	train-error:0.20452	test-error:0.20190                              

[18]	train-error:0.20455	test-error:0.20190                              

[19]	train-error:0.20172	test-error:0.19908                              

[20]	train-error:0.19091	test-error:0.18993                              

[21]	train-error:0.17623	test-error:0.17543                              

[22]	train-error:0.17181	test-error:0.17138                              

[23]	train-error:0.17168	test-error:0.17113                              

[24]	train-error:0.17122	test-error:0.17076                              

[25]	train-error:0.16256	test-error:0.16192                              

[26]	train-error:0.16207	test-error:0.16155                              

[27]	train-error:0.16216	

...

[76]	train-error:0.10436	test-error:0.12629                               

[77]	train-error:0.10396	test-error:0.12654                               

[78]	train-error:0.10362	test-error:0.12641                               

[79]	train-error:0.10353	test-error:0.12623                               

[80]	train-error:0.10332	test-error:0.12617                               

[81]	train-error:0.10322	test-error:0.12611                               

[82]	train-error:0.10261	test-error:0.12598                               

[83]	train-error:0.10258	test-error:0.12611                               

[84]	train-error:0.10215	test-error:0.12617                               

[85]	train-error:0.10206	test-error:0.12617                               

[86]	train-error:0.10175	test-error:0.12598                               

[87]	train-error:0.10129	test-error:0.12629                               

[88]	train-error:0.10086	test-error:0.12654                               

[89]	train-e

[69]	train-error:0.08197	test-error:0.12881                               

[70]	train-error:0.08185	test-error:0.12875                               

[71]	train-error:0.08114	test-error:0.12893                               

[72]	train-error:0.08022	test-error:0.12887                               

[73]	train-error:0.07948	test-error:0.12875                               

[74]	train-error:0.07909	test-error:0.12862                               

[75]	train-error:0.07844	test-error:0.12912                               

[76]	train-error:0.07783	test-error:0.12912                               

[77]	train-error:0.07761	test-error:0.12887                               

[78]	train-error:0.07678	test-error:0.12893                               

[79]	train-error:0.07666	test-error:0.12918                               

[80]	train-error:0.07654	test-error:0.12948                               

[81]	train-error:0.07583	test-error:0.12967                               

[82]	train-e

[62]	train-error:0.12595	test-error:0.12721                               

[63]	train-error:0.12589	test-error:0.12703                               

[64]	train-error:0.12558	test-error:0.12641                               

[65]	train-error:0.12521	test-error:0.12635                               

[66]	train-error:0.12506	test-error:0.12666                               

[67]	train-error:0.12460	test-error:0.12635                               

[68]	train-error:0.12445	test-error:0.12611                               

[69]	train-error:0.12392	test-error:0.12611                               

[70]	train-error:0.12359	test-error:0.12617                               

[71]	train-error:0.12340	test-error:0.12611                               

[72]	train-error:0.12356	test-error:0.12586                               

[73]	train-error:0.12337	test-error:0.12635                               

[74]	train-error:0.12328	test-error:0.12635                               

[75]	train-e

[55]	train-error:0.07497	test-error:0.12954                               

[56]	train-error:0.07494	test-error:0.13047                               

[57]	train-error:0.07393	test-error:0.13096                               

[58]	train-error:0.07291	test-error:0.13059                               

[59]	train-error:0.07230	test-error:0.13102                               

[60]	train-error:0.07144	test-error:0.13139                               

[61]	train-error:0.07119	test-error:0.13157                               

[62]	train-error:0.07070	test-error:0.13237                               

[63]	train-error:0.07024	test-error:0.13219                               

[64]	train-error:0.06953	test-error:0.13170                               

[65]	train-error:0.06864	test-error:0.13182                               

[66]	train-error:0.06812	test-error:0.13249                               

[67]	train-error:0.06723	test-error:0.13243                               

[68]	train-e

[48]	train-error:0.11720	test-error:0.12721                               

[49]	train-error:0.11701	test-error:0.12647                               

[50]	train-error:0.11646	test-error:0.12783                               

[51]	train-error:0.11652	test-error:0.12776                               

[52]	train-error:0.11594	test-error:0.12764                               

[53]	train-error:0.11597	test-error:0.12740                               

[54]	train-error:0.11615	test-error:0.12795                               

[55]	train-error:0.11606	test-error:0.12752                               

[56]	train-error:0.11582	test-error:0.12709                               

[57]	train-error:0.11579	test-error:0.12721                               

[58]	train-error:0.11529	test-error:0.12697                               

[59]	train-error:0.11533	test-error:0.12733                               

[60]	train-error:0.11505	test-error:0.12783                               

[61]	train-e

In [5]:
print(best_val)
print(space_eval(spc, best))

0.122543
{'alpha': 0, 'btype': 'Rn', 'colsample_bylevel': 0.6261447955477483, 'colsample_bytree': 0.5598675876323314, 'eta': 0.06740445691747836, 'eval_metric': ('error',), 'extra_dims': 14, 'gamma': 0, 'lambda': 0.03173665561441266, 'max_depth': 5, 'min_child_weight': 0.08047493164421338, 'objective': 'binary:logistic', 'subsample': 0.9968065361821689}


In [48]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

def objective(param):
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 5,watchlist,evals_result=ed1_results)
    return {'loss': min(ed1_results['test']['error']), 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['binary:logistic']),
    'eval_metric':hp.choice('eval_metric',[['error']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=10)



{'alpha': 2.555696689429851e-07, 'btype': 'R', 'colsample_bylevel': 0.5808673095736296, 'colsample_bytree': 0.7454363436382125, 'eta': 0.09028896383501842, 'eval_metric': ('error',), 'extra_dims': 6, 'gamma': 0, 'lambda': 0.010496473733789414, 'max_depth': 5, 'min_child_weight': 0.003416504180607806, 'objective': 'binary:logistic', 'subsample': 0.6545637422401163}
Overwriting param `num_class`                         
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.               
Setting param `disable_default_eval_metric` to 1.     
[21:57:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

/Users/michaelhorrell/github/tensorflow/wideboost/wideboost/objectives/binarylogloss.py:20: RuntimeWarning: overflow encountered in exp
  P = 1 / (1 + np.exp(-logits))



[2]	train-error:0.17964	test-error:0.17862                            

[3]	train-error:0.29395	test-error:0.29324                            

[4]	train-error:0.32368	test-error:0.32027                            

{'alpha': 1.2189564334503284, 'btype': 'R', 'colsample_bylevel': 0.5154745819953641, 'colsample_bytree': 0.5593323791661391, 'eta': 0.18416194279229148, 'eval_metric': ('error',), 'extra_dims': 15, 'gamma': 0, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 7.906588429791388e-07, 'objective': 'binary:logistic', 'subsample': 0.6948883932449246}
Overwriting param `num_class`                                         
Overwriting param `objective` while setting `obj` in train.           
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                               
Setting param `disable_default_eval_metric` to 1.                     
[21:57:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learne

In [50]:
print(best)
print(space_eval(spc, best))

{'alpha': 1, 'alpha_positive': 2.555696689429851e-07, 'btype': 0, 'colsample_bylevel': 0.5808673095736296, 'colsample_bytree': 0.7454363436382125, 'eta': 0.09028896383501842, 'eval_metric': 0, 'extra_dims': 6, 'gamma': 0, 'lambda': 1, 'lambda_positive': 0.010496473733789414, 'max_depth': 4, 'min_child_weight': 0.003416504180607806, 'objective': 0, 'subsample': 0.6545637422401163}
{'alpha': 2.555696689429851e-07, 'btype': 'R', 'colsample_bylevel': 0.5808673095736296, 'colsample_bytree': 0.7454363436382125, 'eta': 0.09028896383501842, 'eval_metric': ('error',), 'extra_dims': 6, 'gamma': 0, 'lambda': 0.010496473733789414, 'max_depth': 5, 'min_child_weight': 0.003416504180607806, 'objective': 'binary:logistic', 'subsample': 0.6545637422401163}


In [40]:
best

{'btype': 0,
 'colsample_bytree': 0,
 'eta': 0,
 'eval_metric': 0,
 'extra_dims': 4,
 'max_depth': 2,
 'min_child_weight': 0,
 'num_class': 0,
 'objective': 0,
 'subsample': 0}

In [31]:
spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.quniform('extra_dims',0,16,1),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.quniform('max_depth', 2, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

{'learning_rate': <hyperopt.pyll.base.Apply at 0x7fd87547a310>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7fd87547a550>,
 'num_leaves': <hyperopt.pyll.base.Apply at 0x7fd87547a8e0>,
 'min_data_in_leaf': <hyperopt.pyll.base.Apply at 0x7fd875404d00>,
 'feature_fraction': <hyperopt.pyll.base.Apply at 0x7fd8754abb20>,
 'subsample': <hyperopt.pyll.base.Apply at 0x7fd8754abc40>}

In [ ]:
gs_params = {
    'btype':['R','I','Rn','In'],
    'extra_dims':[0,1,4,8,16],
    'max_depth':[1,2,4,8],
    'eta':[0.1],
    'objective':['binary:logistic'],
    'subsample':[1.0],
    'colsample_bytree':[1.0],
    'n_estimators':[50,75],
    'min_child_weight':[1],
    'num_class':[1],
    'eval_metric':[['error']]
}
